In [11]:
import pandas as pd

# 1. 读取数据
df = pd.read_csv('sale_cate.csv')

# 2. 创建“已购买”标记列
df['Bought'] = 1

# 3. 去重，只保留“客户 - 类别”的唯一组合
df_unique = df[['Name', 'Product_Category', 'Bought']].drop_duplicates()

# 4. 构建 客户 x 产品类别 的矩阵
pivot = df_unique.pivot_table(
    index='Name',
    columns='Product_Category',
    values='Bought',
    aggfunc='max',
    fill_value=0
)

# 5. 计算每个产品类别的购买覆盖率（多少客户买过）
category_coverage = pivot.mean().sort_values(ascending=False)

# 6. 设定“热门类别”阈值，例如 80% 客户买过
threshold = 0.75
popular_categories = category_coverage[category_coverage >= threshold].index.tolist()

# 7. 找出这些热门类别中“没买过”的客户
missing_matrix = pivot[popular_categories] == 0

# 8. 整理输出结果：每个热门类别，对应没买过的客户列表
result = {}
for category in popular_categories:
    missing_customers = pivot.index[missing_matrix[category]].tolist()
    if missing_customers:
        result[category] = missing_customers

# 9. 打印结果
for category, customers in result.items():
    print(f"\n🔍 category [{category}] - {len(customers)} customers haven't buy：")
    for customer in customers:
        print(f"   - {customer}")




🔍 category [Bags] - 22 customers haven't buy：
   - 626 Commissary
   - 626 Hospitality Group
   - ALLEN PACKAGING
   - American KGP Inc.
   - And Destroy Coffee
   - BB Bakery
   - Baba Small Batch
   - Beth Jacob Congregation
   - Beyond The Grind
   - Borekas Sephardic Pastries- Van Nuys
   - Borekas Sephardic Pastries- Ventura
   - Bostons Best Coffee
   - Bub and Grandma's Pizza
   - C O D
   - Castanea Sicilian Cafe
   - Cento
   - Cerenzia Foods
   - Cognoscenti Coffee San Julian
   - Couplet Coffee
   - Demitasse Roastery
   - Elat Supermarket
   - F & I COHEN

🔍 category [Packaging] - 24 customers haven't buy：
   - 626 Commissary
   - 626 Hospitality Group
   - A-1 Plus
   - American KGP Inc.
   - And Destroy Coffee
   - Baba Small Batch
   - Beyond The Grind
   - Borekas Sephardic Pastries- Ventura
   - Bostan By Grandma
   - Bostons Best Coffee
   - Bridgetown Roti
   - Bub and Grandma's Pizza
   - Buena Vida Tea Bar
   - C O D
   - Cafe Tondo
   - Carla's Fresh Market
   - 

In [17]:
import pandas as pd

# 1. 读取数据
df = pd.read_csv("Merged_Customer_Category_Table.csv")

# 2. 添加购买标记列
df["Bought"] = 1

# 3. 保留唯一客户 × 品类组合
df_unique = df[["Name", "Product_Category", "Bought"]].drop_duplicates()

# 4. 构建透视表（客户 × 品类）
pivot = df_unique.pivot_table(index='Name', columns='Product_Category', values='Bought', aggfunc='max', fill_value=0)

# 5. 计算每个品类的购买覆盖率（多少客户买过）
category_coverage = pivot.mean()
popular_categories = category_coverage[category_coverage >= 0.76].index.tolist()

# 6. 标记热门品类中哪些客户没买
missing_matrix = pivot[popular_categories] == 0

# 7. 合并客户类型信息
customer_type_map = df[['Name', 'Customer_Category']].drop_duplicates().set_index('Name')['Customer_Category']
pivot['Customer_Type'] = pivot.index.map(customer_type_map)

# 8. 遍历每个客户类型，输出未购买热门品类的客户
result = {}

for ctype in pivot['Customer_Type'].unique():
    group = pivot[pivot['Customer_Type'] == ctype]
    result[ctype] = {}

    for category in popular_categories:
        missing_customers = group.index[group[category] == 0].tolist()
        if missing_customers:
            result[ctype][category] = missing_customers

# 9. 打印结果
for ctype, cat_dict in result.items():
    print(f"\n📂 Customer Type: {ctype}")
    for cat, customers in cat_dict.items():
        print(f"🔸 Category [{cat}] - {len(customers)} customers haven't buy:")
        for name in customers:
            print(f"   - {name}")

# 10. 整理结果为平铺表格格式
output = []

for ctype, cat_dict in result.items():
    for cat, customers in cat_dict.items():
        for name in customers:
            output.append({
                'Customer_Type': ctype,
                'Product_Category': cat,
                'Customer_Name': name
            })

# 转换为 DataFrame
df_output = pd.DataFrame(output)

# 导出为 Excel 文件
df_output.to_excel("Unpurchased_Popular_Categories.xlsx", index=False)
print("✅ 导出完成：Unpurchased_Popular_Categories.xlsx")



📂 Customer Type: Restaurant – Vietnamese

📂 Customer Type: Restaurant – Contemporary American
🔸 Category [Bags] - 1 customers haven't buy:
   - 626 Commissary
🔸 Category [Packaging] - 1 customers haven't buy:
   - 626 Commissary

📂 Customer Type: Hospitality Group
🔸 Category [Bags] - 1 customers haven't buy:
   - 626 Hospitality Group
🔸 Category [Packaging] - 1 customers haven't buy:
   - 626 Hospitality Group

📂 Customer Type: Services – General
🔸 Category [Packaging] - 1 customers haven't buy:
   - A-1 Plus

📂 Customer Type: Manufacturing – Packaging
🔸 Category [Bags] - 1 customers haven't buy:
   - ALLEN PACKAGING
🔸 Category [Lids] - 1 customers haven't buy:
   - ALLEN PACKAGING

📂 Customer Type: Hotel
🔸 Category [Lids] - 1 customers haven't buy:
   - Ace Hotel

📂 Customer Type: Industrial Supplier
🔸 Category [Bags] - 1 customers haven't buy:
   - American KGP Inc.
🔸 Category [Lids] - 2 customers haven't buy:
   - American KGP Inc.
   - Dura Sourcing, INC
🔸 Category [Packaging] - 2

In [18]:
import pandas as pd

# 1. 读取数据
df = pd.read_csv("Merged_Customer_Category_Table.csv")
df["Bought"] = 1

# 2. 获取客户-品类唯一组合
df_unique = df[["Name", "Product_Category", "Bought"]].drop_duplicates()

# 3. 透视表（客户 × 品类）
pivot = df_unique.pivot_table(index='Name', columns='Product_Category', values='Bought', aggfunc='max', fill_value=0)

# 4. 品类覆盖率
category_coverage = pivot.mean()
popular_categories = category_coverage[category_coverage >= 0.76]

# 5. 客户类型信息
customer_type_map = df[['Name', 'Customer_Category']].drop_duplicates().set_index('Name')['Customer_Category']
pivot['Customer_Type'] = pivot.index.map(customer_type_map)

# 6. 构建完整行级表格
output = []

for name, row in pivot.iterrows():
    ctype = row["Customer_Type"]
    for cat in popular_categories.index:
        purchased = row[cat]
        output.append({
            "Customer_Name": name,
            "Customer_Type": ctype,
            "Product_Category": cat,
            "Purchased": int(purchased),
            "Category_Coverage": round(popular_categories[cat], 2)
        })

# 7. 转为 DataFrame 并导出
df_output = pd.DataFrame(output)
df_output.to_excel("Customer_Unpurchased_Report.xlsx", index=False)
print("✅ 导出完成：Customer_Unpurchased_Report.xlsx")


✅ 导出完成：Customer_Unpurchased_Report.xlsx
